In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import pickle
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-Retriever


In [3]:
with open("results/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [4]:
split_documents

[Document(metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': []}, page_content='OECD\n\nOECD\n\nOECD Economic Outlook, Interim Report\n=====================================\n\n  \n\nFinding the Right Balance in Uncertain Times\n============================================\n\nSeptember 2025\n=============='),
 Document(metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': []}, page_content='OECD Economic Outlook, Interim Report\n=====================================\n\nOECD Economic Outlook, Interim Report\n=====================================\n\nFinding the Right Balance in Uncertain Times\n============================================\n\nSeptember 2025\n==============\n\nOECD  \nBETTER POLICIES FOR BETTER LIVES'),
 Document(metadata={'page': 4, 'image_id': [], 'image_path': [], 'text_summary': '- The text is a copyright, licensing, and disclaimer notice for the OECD Economic Outlook, Interim Report Septembe

In [6]:
len(split_documents)

109

In [5]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [7]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [12]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [13]:
# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 3

In [14]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

In [15]:
# 검색 결과 문서를 가져옵니다.
query = "what are risks and challenges in global economy?"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Finding the right balance in uncertain times 4  
Summary 4  
Recent developments 7  
Projections 14  
Risks and challenges 16  
Policy requirements 18  
FIGURES  
Figure 1. Global growth has remained resilient with front-loading supporting industrial production 8  
Figure 2. United States trade growth soared prior to April but has since weakened 8  
Figure 3. Industrial production and retail sales growth are easing in several major economies 9  
Figure 4. Higher tariff rates are gradually feeding through into the real economy 11  
Figure 5. Labour markets have softened in some countries 12  
Figure 6. Disinflation has stalled in some OECD economies 13  
Figure 7. Heightened risk appetite is coupled with growing concerns about future fiscal risks 14  
Figure 8. Economic growth is projected to weaken in G20 economies 16  
Figure 9. Valuations of technology stocks and crypto-assets are elevated 17  
Figure 10. Policy rates are expected to ease gradually 19

C

In [16]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [18]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "글로벌 경제에서 위험 및 도전 요소는 무엇인가?"
response = chain.invoke(question)
print(response)

주어진 보고서를 바탕으로 한 주요 위험·도전 요소는 다음과 같습니다.

- 무역장벽 확대 및 관세 인상: 관세 상승과 무역제한 조치가 투자·교역을 위축시키고 성장 하방 압력을 높임.  
- 무역 분절(프래그먼테이션)과 공급망 취약성: 국가별 규제·정책 분화로 공급망 불안정이 확대될 위험.  
- 인플레이션 재확산 가능성: 물가 상승 압력이 재급증하면 통화정책 불확실성과 경기 둔화 유발.  
- 재정 리스크 확대: 재정건전성 우려와 채무 지속가능성 문제가 정책 대응 능력을 약화시킬 수 있음.  
- 금융시장 리프라이싱(재평가) 및 자산버블: 기술주·암호자산 등 고평가 자산의 조정이 금융안정성을 위협.  
- 투자·생산성 부진의 장기화: 약한 투자와 낮은 생산성 성장이 잠재성장률을 낮춤.  
- 노동력 부족 및 노동시장 약화: 인력 부족·참여율 정체(특히 여성 고용 개선 필요)가 성장 제약.  
- 통화정책·금리 불확실성: 물가 기대나 경기 전개에 따라 금리 경로가 불확실해짐.  
- 신기술(예: AI) 확산의 불확실성 및 불균형적 효과: AI는 성장 기회이나 도입 속도·인력·정책에 따라 편차가 커 사회·경제적 도전 초래.

이들 위험은 상호작용하면서 글로벌 성장과 물가·금융안정에 복합적인 하방 리스크를 제공한다.
